In [1]:
import ee

ee.Authenticate()
ee.Initialize(project="biodiversitymeetsdata")

ModuleNotFoundError: No module named 'ee'

In [2]:
def bbox_to_rect(bbox_dict):
    return ee.Geometry.Rectangle([
        bbox_dict["west"], bbox_dict["south"],
        bbox_dict["east"], bbox_dict["north"]
    ])

meiseSpatial = {"west": 4.312323, "south": 50.922796, "east": 4.335497, "north": 50.934860}
meise_roi = bbox_to_rect(meiseSpatial)

# Extracting data

## Image

Raster data are represented as Image objects in Earth Engine. Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image has metadata stored as a set of properties.

In [20]:
S2_img = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(meise_roi).filterDate("2020-01-01", "2020-01-31").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
#Select the appropriate bands 
bands=["B4","B3","B2"]

S2_img = S2_img.sort('CLOUDY_PIXEL_PERCENTAGE').first()
B432_band = S2_img.select(*bands)

#geometries=True is included to associate each poxel with a geographic location
result=B432_band.sampleRegions(collection=ee.FeatureCollection([meise_roi]), scale=30, geometries=True).getInfo()
#Returns an json style object 
"""
{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B4', 'B3', 'B2']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [9.950032251174557, 37.93924767221463]},
   'id': '0_0',
   'properties': {'B2': 5898, 'B3': 5376, 'B4': 5003}},...]}
"""

"\n{'type': 'FeatureCollection',\n 'columns': {},\n 'properties': {'band_order': ['B4', 'B3', 'B2']},\n 'features': [{'type': 'Feature',\n   'geometry': {'geodesic': False,\n    'type': 'Point',\n    'coordinates': [9.950032251174557, 37.93924767221463]},\n   'id': '0_0',\n   'properties': {'B2': 5898, 'B3': 5376, 'B4': 5003}},...]}\n"

In [13]:
import itertools

def extractRow(featureRow, bands):
    """
    Extracts all relevant information from a pixel contained within the JSON object
    
    Args:
        featureRow (dict): A dictionary which is part of the jsonObj["features"] list
        bands (list): A list containing the bands of interest that are selected from the Image instance
        
    Returns:
        valueDict (dict): A dictionary where the longitude, latitude and bands are mapped to their corresponding 
                          values in the json object from the sampleRegions function
    """
    featureKeys=["longitude", "latitude", *bands]
    values = list(itertools.chain([featureRow["geometry"]["coordinates"][0], featureRow["geometry"]["coordinates"][1]],
                                      [featureRow["properties"][key] for key in bands]))
    return dict(zip(featureKeys, values))

import pandas as pd

def jsonToDf(jsonObj, bands):
    data=[extractRow(featureRow, bands) for featureRow in jsonObj["features"]]
    return pd.DataFrame(data)

In [14]:
df=jsonToDf(result, bands)
df.head()

,longitude,latitude,B4,B3,B2
0,4.315775,50.934856,3365,3434,3608
1,4.316202,50.934851,2861,2983,3226
2,4.316629,50.934846,3293,3266,3384
3,4.317056,50.934842,1897,1934,2113
4,4.317483,50.934837,1888,1931,2038


In [23]:
import folium 

visualParam = {'min':0,
               'max':4000,
               'bands':["B4", "B3", "B2"]}

map_center = [50.9288, 4.324]
m = folium.Map(location=map_center, zoom_start=14)

map_id_dict = ee.Image(B432_band).getMapId(visualParam)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Sentinel-2 RGB',
    overlay=True,
    control=True
).add_to(m)

In [24]:
m

## Image Collection

In [27]:
S2_img = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate("2020-01-01", "2020-01-02")
#S2_img
B432_band = S2_img.select("B4","B3","B2") #Select the red blue and green band
B432_band.getRegion(meise_roi, scale=10).getInfo()

[['id', 'longitude', 'latitude', 'time', 'B4', 'B3', 'B2'],
 ['20200101T105441_20200101T105437_T31UES',
  4.312407437179969,
  50.92284342664752,
  1577876210178,
  1034,
  1140,
  1229],
 ['20200101T105441_20200101T105437_T31UES',
  4.3124972687083805,
  50.92284342664752,
  1577876210178,
  1034,
  1140,
  1229],
 ['20200101T105441_20200101T105437_T31UES',
  4.312587100236793,
  50.92284342664752,
  1577876210178,
  1066,
  1106,
  1179],
 ['20200101T105441_20200101T105437_T31UES',
  4.312676931765204,
  50.92284342664752,
  1577876210178,
  1160,
  1208,
  1300],
 ['20200101T105441_20200101T105437_T31UES',
  4.312766763293617,
  50.92284342664752,
  1577876210178,
  1160,
  1208,
  1300],
 ['20200101T105441_20200101T105437_T31UES',
  4.312856594822029,
  50.92284342664752,
  1577876210178,
  1126,
  1196,
  1284],
 ['20200101T105441_20200101T105437_T31UES',
  4.3129464263504405,
  50.92284342664752,
  1577876210178,
  1126,
  1196,
  1284],
 ['20200101T105441_20200101T105437_T31UES'

# Computing new layers

In [28]:
S2_img = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(meise_roi).filterDate("2020-01-01", "2020-01-31").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
#Select the appropriate bands 
bands=["B4","B3","B2"]

S2_img = S2_img.sort('CLOUDY_PIXEL_PERCENTAGE').first()
B432_band = S2_img.select(*bands)

#geometries=True is included to associate each poxel with a geographic location
result=B432_band.sampleRegions(collection=ee.FeatureCollection([meise_roi]), scale=30, geometries=True).getInfo()
#Returns an json style object 
"""
{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B4', 'B3', 'B2']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [9.950032251174557, 37.93924767221463]},
   'id': '0_0',
   'properties': {'B2': 5898, 'B3': 5376, 'B4': 5003}},...]}
"""

"\n{'type': 'FeatureCollection',\n 'columns': {},\n 'properties': {'band_order': ['B4', 'B3', 'B2']},\n 'features': [{'type': 'Feature',\n   'geometry': {'geodesic': False,\n    'type': 'Point',\n    'coordinates': [9.950032251174557, 37.93924767221463]},\n   'id': '0_0',\n   'properties': {'B2': 5898, 'B3': 5376, 'B4': 5003}},...]}\n"

In [29]:
# Initial date of interest (inclusive).
i_date = "2015-01-01"

# Final date of interest (exclusive).
f_date = "2020-01-01"

# Define the location of interest with a point.
lon = 5.145041
lat = 45.772439
poi = ee.Geometry.Point(lon, lat)

# A nominal scale in meters of the projection to work in [in meters].
scale = 1000

In [30]:
# Soil depths [in cm] where we have data.
olm_depths = [0, 10, 30, 60, 100, 200]

# Names of bands associated with reference depths.
olm_bands = ["b" + str(sd) for sd in olm_depths]

In [31]:
def get_soil_prop(param):
    """
    This function returns soil properties image
    param (str): must be one of:
        "sand"     - Sand fraction
        "clay"     - Clay fraction
        "orgc"     - Organic Carbon fraction
    """
    if param == "sand":  # Sand fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "clay":  # Clay fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "orgc":  # Organic Carbon fraction [g/kg]
        snippet = "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 5 * 0.001  # to get kg/kg
    else:
        return print("error")

    # Apply the scale factor to the ee.Image.
    dataset = ee.Image(snippet).multiply(scale_factor)

    return dataset

In [32]:
# Image associated with the sand content.
sand = get_soil_prop("sand")

# Image associated with the clay content.
clay = get_soil_prop("clay")

# Image associated with the organic carbon content.
orgc = get_soil_prop("orgc")

In [33]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)


# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

my_map = folium.Map(location=[lat, lon], zoom_start=3)

# Set visualization parameters.
vis_params = {
    "bands": ["b0"],
    "min": 0.01,
    "max": 1,
    "opacity": 1,
    "palette": ["white", "#464646"],
}

# Add the sand content data to the map object.
my_map.add_ee_layer(sand, vis_params, "Sand Content")

# Add a marker at the location of interest.
folium.Marker([lat, lon], popup="point of interest").add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)